### New low dimensional feature matrix
- Binary classifier vs 3-class classification <br>
> Binary: AMT rating increases(pos) vs others(neg)<br>
> 3-class: AMT rating increases(1), keep same(0), decreases(-1) <br>

- Coefficient of source word and target word <br>
- Number of support words and opposite words abs(coef) > 0.1 <br>
- Sentence length <br>
- Depth of source word and target word (VT,CTF,CSF) <br>
- Length of path for source sentence and target sentence <br>

In [64]:
import pickle, pydot, random, re, ast, time
import pandas as pd
import numpy as np
import ast
from scipy.sparse import vstack
from sklearn.externals.six import StringIO 
from IPython.display import display
#from sklearn.datasets import load_iris
from sklearn import tree
from IPython.display import Image
from collections import Counter
import seaborn as sns
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
project_path = '/data/2/zwang/2018_S_WordTreatment/'
amt_path = '/data/2/zwang/2018_S_WordTreatment/V2_AMT/6_amt/'
dataset_list = ['yp','tw','airbnb']
methods = ['KNN','VT','CTF','CSF']
full_name = {'yp':'Yelp','tw':'Twitter','airbnb':'Airbnb'}
data_file = {}
data_file['yp'] = project_path + 'V2_yelp/5_Select/yp_wdpair_sents_4effects_posinters_poscom1_bigramcheck_limit.pickle'
data_file['tw'] = project_path + 'V2_twitter/5_Select/tw_wdpair_sents_4effects_posinters_poscom1_bigramcheck_limit.pickle'
data_file['airbnb'] = project_path + 'V2_airbnb/5_Select/airbnb_wdpair_sents_4effects_posinters_poscom1_bigramcheck_limit_limitvocab.pickle'
vocab_file = project_path+'V2_airbnb/0_Data/common_wds.csv'

In [3]:
def cal_causal_words(data_pd,pos_thresh,neg_thresh):
    n_sup = []
    n_opp = []
    for sent in data_pd.sorted_sentence.values:
        nsp = 0
        nop = 0
        for item in ast.literal_eval(sent):
            if(item[1] > pos_thresh):
                nsp += 1
            elif(item[1] < neg_thresh):
                nop += 1
        n_sup.append(nsp)
        n_opp.append(nop)
    data_pd['n_sup'] = n_sup
    data_pd['n_opp'] = n_opp
    return data_pd

In [295]:
new_feats = ['src_coef','tar_coef','n_sup','n_opp','n_words','src_probLR','tar_probLR',
             'vt_srcwd_depth','vt_tarwd_depth','vt_src_pathlen','vt_tar_pathlen',
             'ctf_srcwd_depth','ctf_tarwd_depth','ctf_src_pathlen','ctf_tar_pathlen']

In [4]:
new_feats = ['tar_src_dif_coef','n_sup','n_opp','n_words','src_probLR']

In [5]:
def cal_newfeats(coef_pd,lr_file,vt_file,ctf_file):
    lr_pd = pd.read_csv(lr_file)
    lr_pd['n_words'] = lr_pd['src_sentence'].apply(lambda x: len(re.findall('\w+',x)))
    lr_pd['src_probLR'] = lr_pd['src_prob']
    lr_pd['tar_probLR'] = lr_pd['tar_prob']
    vt_pd = pd.read_csv(vt_file)
    vt_pd['vt_srcwd_depth'] = vt_pd['src_depth'].apply(lambda x: np.mean(ast.literal_eval(x)))
    vt_pd['vt_tarwd_depth'] = vt_pd['tar_depth'].apply(lambda x: np.mean(ast.literal_eval(x)))
    vt_pd['vt_src_pathlen'] = vt_pd['src_path_len'].apply(lambda x: np.mean(ast.literal_eval(x)))
    vt_pd['vt_tar_pathlen'] = vt_pd['tar_path_len'].apply(lambda x: np.mean(ast.literal_eval(x)))
    ctf_pd = pd.read_csv(ctf_file)
    ctf_pd['ctf_srcwd_depth'] = ctf_pd['src_depth'].apply(lambda x: np.mean(ast.literal_eval(x)))
    ctf_pd['ctf_tarwd_depth'] = ctf_pd['tar_depth'].apply(lambda x: np.mean(ast.literal_eval(x)))
    ctf_pd['ctf_src_pathlen'] = ctf_pd['src_path_len'].apply(lambda x: np.mean(ast.literal_eval(x)))
    ctf_pd['ctf_tar_pathlen'] = ctf_pd['tar_path_len'].apply(lambda x: np.mean(ast.literal_eval(x)))
            
    '''
    Has formatting problem when matching, make error on cases like 0.26666666666 and 0.26700000001.
    '''
    coef_lr_pd = coef_pd.merge(lr_pd,on=['id','source','target'])
    vt_ctf_pd = vt_pd.merge(ctf_pd,on=['id','source','target']) 
    super_pd = coef_lr_pd.merge(vt_ctf_pd,on=['id','source','target'])
    
    for col in ['amt_effect_mostcom','knn_effect','vt_effect','ctf_effect','csf_effect']:
        super_pd[col]=super_pd[col+'_x_x']
    return super_pd

In [6]:
def fit_LR_newfeats(dataset_list,new_feats,clf_flag):
    for di in range(len(dataset_list)):
        print("Processing %s ..." % (full_name[dataset_list[di]]))
        coef_file = project_path+'V2_'+full_name[dataset_list[di]].lower()+'/9_Coef/'+dataset_list[di]+'_sentence_coef.csv'
        org_coef_pd = pd.read_csv(coef_file)
        org_coef_pd['tar_src_dif_coef'] = org_coef_pd['tar_coef'] - org_coef_pd['src_coef'] 
        coef_pd = cal_causal_words(org_coef_pd,0.1,-0.1)
        lr_file = project_path+'V2_'+full_name[dataset_list[di]].lower()+'/8_Var/'+dataset_list[di]+'_effectVar_prob_byLR.csv'
        vt_file = project_path+'V2_'+full_name[dataset_list[di]].lower()+'/8_Var/'+dataset_list[di]+'_VT_srctar_depth3.csv'
        ctf_file = project_path+'V2_'+full_name[dataset_list[di]].lower()+'/8_Var/'+dataset_list[di]+'_CTF_srctar_depth.csv'
        super_pd = cal_newfeats(coef_pd,lr_file,vt_file,ctf_file)
        print(super_pd.shape)
        
        # construct feature matrix
        X_feats = []
        for i in range(super_pd.shape[0]):
            row = super_pd.iloc[i][new_feats].values
            X_feats.append([float(e) for e in row])
        
        # fit logistic-regression or linear-regression
        if(clf_flag == 'binary'):
            LR_clf = LogisticRegression(random_state=42,n_jobs=-1)
            amt_labels = super_pd['amt_effect_mostcom'].apply(lambda x: 1 if x>0 else 0)
            LR_clf.fit(X_feats,amt_labels)
            print(Counter(amt_labels),'%.3f' % (np.mean(cross_val_score(LR_clf, X_feats, amt_labels, cv=3))))
            for fi in LR_clf.coef_[0].argsort()[::-1]:
                print("%.3f , %s" % (LR_clf.coef_[0][fi],new_feats[fi]))
        
        elif(clf_flag == '3-class'):
            LR_clf = LogisticRegression(random_state=42,n_jobs=-1)
            amt_labels = super_pd['amt_effect_mostcom'].apply(lambda x: 1 if x>0 else (-1 if x<0 else 0))
            LR_clf.fit(X_feats,amt_labels)
            print(Counter(amt_labels),'%.3f' % (np.mean(cross_val_score(LR_clf, X_feats, amt_labels, cv=3))))
            for cls in LR_clf.classes_:
                print("class:",cls)
                ci = LR_clf.classes_[cls]
                for fi in LR_clf.coef_[ci].argsort()[::-1]:
                    print("%.3f , %s" % (LR_clf.coef_[ci][fi],new_feats[fi])) # 3-class
            
        elif(clf_flag == 'linear'):
            for md in ['knn_effect','vt_effect','ctf_effect','csf_effect']:
                print(md)
                LR_clf = LinearRegression(n_jobs=-1)
                LR_clf.fit(X_feats,super_pd[md].values)

                for idx in LR_clf.coef_.argsort()[::-1]:
                    print("%.3f , %s " % (LR_clf.coef_[idx], new_feats[idx]))

#### Result for 15 new features 

In [303]:
fit_LR_newfeats(dataset_list,new_feats,clf_flag='3-class')

Processing Yelp ...
(120, 95)
Counter({1: 61, 0: 41, -1: 18}) 0.567
class: -1
0.146 , src_probLR
0.104 , src_coef
0.087 , n_sup
0.081 , vt_src_pathlen
0.073 , n_opp
0.054 , ctf_tar_pathlen
-0.010 , n_words
-0.027 , ctf_src_pathlen
-0.040 , vt_srcwd_depth
-0.050 , ctf_tarwd_depth
-0.050 , ctf_srcwd_depth
-0.069 , vt_tar_pathlen
-0.070 , tar_probLR
-0.105 , vt_tarwd_depth
-0.602 , tar_coef
class: 0
0.874 , ctf_tarwd_depth
0.874 , ctf_srcwd_depth
0.799 , tar_coef
0.195 , n_sup
0.158 , vt_tarwd_depth
0.039 , ctf_src_pathlen
0.018 , vt_src_pathlen
0.003 , ctf_tar_pathlen
-0.020 , n_opp
-0.022 , vt_tar_pathlen
-0.047 , vt_srcwd_depth
-0.104 , n_words
-0.132 , tar_probLR
-0.196 , src_coef
-0.421 , src_probLR
class: 1
0.255 , vt_tar_pathlen
0.222 , src_probLR
0.204 , n_words
0.143 , vt_srcwd_depth
0.084 , tar_probLR
0.025 , src_coef
-0.020 , vt_tarwd_depth
-0.066 , ctf_src_pathlen
-0.114 , n_opp
-0.164 , ctf_tar_pathlen
-0.267 , vt_src_pathlen
-0.365 , tar_coef
-0.540 , n_sup
-0.640 , ctf_tarw

In [304]:
fit_LR_newfeats(dataset_list,new_feats,clf_flag='binary')

Processing Yelp ...
(120, 95)
Counter({1: 61, 0: 59}) 0.651
0.874 , ctf_tarwd_depth
0.874 , ctf_srcwd_depth
0.799 , tar_coef
0.195 , n_sup
0.158 , vt_tarwd_depth
0.039 , ctf_src_pathlen
0.018 , vt_src_pathlen
0.003 , ctf_tar_pathlen
-0.020 , n_opp
-0.022 , vt_tar_pathlen
-0.047 , vt_srcwd_depth
-0.104 , n_words
-0.132 , tar_probLR
-0.196 , src_coef
-0.421 , src_probLR
Processing Twitter ...
(120, 95)
Counter({0: 63, 1: 57}) 0.533
0.546 , tar_probLR
0.187 , vt_srcwd_depth
0.140 , tar_coef
0.033 , n_words
0.007 , vt_tar_pathlen
0.005 , vt_tarwd_depth
-0.012 , vt_src_pathlen
-0.036 , ctf_src_pathlen
-0.037 , n_sup
-0.050 , ctf_tar_pathlen
-0.212 , src_coef
-0.263 , n_opp
-0.324 , ctf_tarwd_depth
-0.324 , ctf_srcwd_depth
-0.995 , src_probLR
Processing Airbnb ...
(120, 95)
Counter({0: 86, 1: 34}) 0.582
0.481 , tar_coef
0.308 , vt_src_pathlen
0.307 , ctf_tarwd_depth
0.307 , ctf_srcwd_depth
0.139 , n_opp
0.040 , vt_tarwd_depth
0.018 , n_sup
-0.028 , n_words
-0.059 , vt_srcwd_depth
-0.072 , ct

In [305]:
fit_LR_newfeats(dataset_list,new_feats,clf_flag='linear')

Processing Yelp ...
(120, 95)
knn_effect
0.070 , tar_coef 
0.009 , n_words 
0.008 , ctf_src_pathlen 
0.006 , vt_tar_pathlen 
0.002 , vt_src_pathlen 
0.002 , ctf_tar_pathlen 
0.000 , ctf_srcwd_depth 
0.000 , ctf_tarwd_depth 
-0.002 , n_sup 
-0.010 , vt_srcwd_depth 
-0.012 , n_opp 
-0.024 , vt_tarwd_depth 
-0.031 , src_coef 
-0.075 , src_probLR 
-0.085 , tar_probLR 
vt_effect
0.141 , tar_probLR 
0.033 , tar_coef 
0.003 , vt_tar_pathlen 
0.002 , vt_srcwd_depth 
0.000 , ctf_src_pathlen 
0.000 , vt_src_pathlen 
0.000 , ctf_tarwd_depth 
-0.000 , ctf_srcwd_depth 
-0.000 , n_opp 
-0.001 , n_words 
-0.002 , ctf_tar_pathlen 
-0.004 , n_sup 
-0.004 , src_coef 
-0.006 , vt_tarwd_depth 
-0.151 , src_probLR 
ctf_effect
0.262 , tar_probLR 
0.070 , tar_coef 
0.006 , vt_tar_pathlen 
0.005 , n_opp 
0.004 , n_sup 
0.003 , ctf_src_pathlen 
0.003 , vt_srcwd_depth 
0.000 , ctf_tarwd_depth 
-0.000 , ctf_srcwd_depth 
-0.003 , ctf_tar_pathlen 
-0.003 , vt_src_pathlen 
-0.004 , n_words 
-0.011 , vt_tarwd_depth 

In [62]:
#vt_pd['src_depth'].apply(lambda x: np.mean(ast.literal_eval(x)))

#### Result for 5 new features

In [7]:
fit_LR_newfeats(dataset_list,new_feats,clf_flag='3-class')

Processing Yelp ...
(120, 96)
Counter({1: 61, 0: 41, -1: 18}) 0.616
class: -1
0.087 , n_opp
0.084 , n_sup
0.005 , src_probLR
-0.036 , n_words
-0.334 , tar_src_dif_coef
class: 0
0.530 , tar_src_dif_coef
0.156 , n_sup
-0.075 , n_words
-0.077 , n_opp
-0.569 , src_probLR
class: 1
0.343 , src_probLR
0.205 , n_words
-0.074 , n_opp
-0.450 , n_sup
-0.528 , tar_src_dif_coef
Processing Twitter ...
(120, 96)
Counter({1: 57, 0: 45, -1: 18}) 0.425
class: -1
0.318 , src_probLR
0.291 , n_opp
0.149 , n_sup
-0.131 , n_words
-0.275 , tar_src_dif_coef
class: 0
0.160 , tar_src_dif_coef
0.028 , n_words
0.002 , n_sup
-0.178 , n_opp
-0.521 , src_probLR
class: 1
0.134 , n_words
0.061 , tar_src_dif_coef
-0.001 , src_probLR
-0.168 , n_opp
-0.238 , n_sup
Processing Airbnb ...
(120, 96)
Counter({0: 73, 1: 34, -1: 13}) 0.600
class: -1
1.050 , src_probLR
0.017 , n_words
-0.028 , n_opp
-0.110 , n_sup
-0.165 , tar_src_dif_coef
class: 0
0.240 , tar_src_dif_coef
0.112 , n_opp
0.022 , n_sup
-0.028 , n_words
-0.797 , src

In [8]:
fit_LR_newfeats(dataset_list,new_feats,clf_flag='binary')

Processing Yelp ...
(120, 96)
Counter({1: 61, 0: 59}) 0.734
0.530 , tar_src_dif_coef
0.156 , n_sup
-0.075 , n_words
-0.077 , n_opp
-0.569 , src_probLR
Processing Twitter ...
(120, 96)
Counter({0: 63, 1: 57}) 0.533
0.160 , tar_src_dif_coef
0.028 , n_words
0.002 , n_sup
-0.178 , n_opp
-0.521 , src_probLR
Processing Airbnb ...
(120, 96)
Counter({0: 86, 1: 34}) 0.717
0.240 , tar_src_dif_coef
0.112 , n_opp
0.022 , n_sup
-0.028 , n_words
-0.797 , src_probLR


In [9]:
fit_LR_newfeats(dataset_list,new_feats,clf_flag='linear')

Processing Yelp ...
(120, 96)
knn_effect
0.034 , tar_src_dif_coef 
-0.001 , n_words 
-0.005 , n_sup 
-0.009 , n_opp 
-0.189 , src_probLR 
vt_effect
0.031 , src_probLR 
0.018 , tar_src_dif_coef 
0.002 , n_opp 
-0.001 , n_sup 
-0.004 , n_words 
ctf_effect
0.079 , src_probLR 
0.044 , tar_src_dif_coef 
0.015 , n_opp 
0.014 , n_sup 
-0.013 , n_words 
csf_effect
0.086 , src_probLR 
0.041 , tar_src_dif_coef 
0.015 , n_opp 
0.006 , n_sup 
-0.006 , n_words 
Processing Twitter ...
(120, 96)
knn_effect
0.018 , tar_src_dif_coef 
0.016 , n_words 
-0.021 , n_sup 
-0.046 , n_opp 
-0.453 , src_probLR 
vt_effect
0.024 , tar_src_dif_coef 
0.010 , n_sup 
0.002 , n_opp 
-0.009 , n_words 
-0.075 , src_probLR 
ctf_effect
0.022 , tar_src_dif_coef 
0.022 , n_sup 
0.005 , n_opp 
-0.009 , n_words 
-0.294 , src_probLR 
csf_effect
0.027 , tar_src_dif_coef 
0.016 , n_sup 
0.006 , n_opp 
-0.006 , n_words 
-0.164 , src_probLR 
Processing Airbnb ...
(120, 96)
knn_effect
0.018 , n_sup 
0.014 , tar_src_dif_coef 
0.009 

### Predict positive probability for each instance and then check correlation between P(y=1|x) and each treatment effect(KNN,VT,CTF,CSF).

In [62]:
def predict_prob_newfeats(dataset_list,new_feats,LSE_effect,flag):
    """
    Predict p(y=1|x) with new features.
    In-bag prediction
    """
    all_prob_corrs = []
    for di in range(len(dataset_list)):
        print("Processing %s ..." % (full_name[dataset_list[di]]))
        coef_file = project_path+'V2_'+full_name[dataset_list[di]].lower()+'/9_Coef/'+dataset_list[di]+'_sentence_coef.csv'
        org_coef_pd = pd.read_csv(coef_file)
        org_coef_pd['tar_src_dif_coef'] = org_coef_pd['tar_coef'] - org_coef_pd['src_coef'] 
        coef_pd = cal_causal_words(org_coef_pd,0.1,-0.1)
        lr_file = project_path+'V2_'+full_name[dataset_list[di]].lower()+'/8_Var/'+dataset_list[di]+'_effectVar_prob_byLR.csv'
        vt_file = project_path+'V2_'+full_name[dataset_list[di]].lower()+'/8_Var/'+dataset_list[di]+'_VT_srctar_depth3.csv'
        ctf_file = project_path+'V2_'+full_name[dataset_list[di]].lower()+'/8_Var/'+dataset_list[di]+'_CTF_srctar_depth.csv'
        super_pd = cal_newfeats(coef_pd,lr_file,vt_file,ctf_file)
        print(super_pd.shape)
        
        # construct feature matrix
        X_feats = []
        for i in range(super_pd.shape[0]):
            row = super_pd.iloc[i][new_feats].values
            X_feats.append([float(e) for e in row])
        X_feats = np.array(X_feats)
        
        # fit binary logistic-regression
        LR_clf = LogisticRegression(random_state=42,n_jobs=-1)
        amt_labels = super_pd[AMT_method].apply(lambda x: 1 if x>0 else 0)
        if(flag=='inb'):
            LR_clf.fit(X_feats,amt_labels)
            pos_prob = LR_clf.predict_proba(X_feats)[:,1]
        elif(flag=='oob'):
            pos_prob=[]
            for ti in range(X_feats.shape[0]):
                sel_idx = [idx for idx in range(X_feats.shape[0]) if idx != ti]
                LR_clf.fit(X_feats[sel_idx,:],amt_labels[sel_idx])
                pos_prob.append(LR_clf.predict_proba(X_feats[ti])[0,1])
                
        prob_corrs = []
        for eft in LSE_effect:
            prob_corrs.append(pearsonr(pos_prob,super_pd[eft])[0])
        all_prob_corrs.append(pd.DataFrame(prob_corrs,index=LSE_effect,columns=[full_name[dataset_list[di]]]))
        
        
    display(pd.concat(all_prob_corrs,axis=1))
        

In [49]:
LSE_effect = ['knn_effect','vt_effect','ctf_effect','csf_effect']
predict_prob_newfeats(dataset_list,new_feats,LSE_effect,AMT_method = 'amt_effect_mostcom',flag='inbag')

Processing Yelp ...
(120, 96)
Processing Twitter ...
(120, 96)
Processing Airbnb ...
(120, 96)


,Yelp,Twitter,Airbnb
knn_effect,0.365381,0.341582,0.452208
vt_effect,0.622568,0.412877,0.304252
ctf_effect,0.550506,0.217210,0.490342
csf_effect,0.387922,0.134598,0.276877


In [65]:
LSE_effect = ['knn_effect','vt_effect','ctf_effect','csf_effect']
predict_prob_newfeats(dataset_list,new_feats,LSE_effect,flag='oob')

Processing Yelp ...
(120, 96)
Processing Twitter ...
(120, 96)
Processing Airbnb ...
(120, 96)


,Yelp,Twitter,Airbnb
knn_effect,0.366059,0.284777,0.464504
vt_effect,0.617978,0.381676,0.315482
ctf_effect,0.544271,0.136027,0.500171
csf_effect,0.384367,0.112451,0.279512
